# ADC Feature Analysis

In [ ]:
import SimpleITK as sitk
import numpy as np
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from skimage import measure
from skimage.measure import find_contours

In [ ]:
feature_file = 'proviz_feature_outout'
#Importing and reading image
feature_image = nib.load(feature_file)
feature_data = feature_image.get_fdata()

num_slices = feature_data.shape[2]
ROI_mask = (feature_data != 0)

In [ ]:
slice_stats = []

for slice_idx in range(num_slices):
    slice_data = feature_data[:, :, slice_idx]
    min_val = np.min(slice_data)
    mean_val = np.mean(ROI_mask[..., slice_idx])
    min_loc = np.unravel_index(np.argmin(slice_data), slice_data.shape)
    slice_stats.append((slice_idx, min_val, mean_val, min_loc))


stats_df = pd.DataFrame(slice_stats, columns=['Slice Index', 'Minimum Value', 'Mean Value', 'Min Location'])

cols = 5
rows = int(np.ceil(num_slices / cols))


fig, axes = plt.subplots(rows, cols, figsize=(15, rows * 3))
axes = axes.flatten()  #Flatten axes for easy iteration

for slice_idx in range(num_slices):
    slice_data = feature_data[:, :, slice_idx]
    min_loc = np.unravel_index(np.argmin(slice_data), slice_data.shape)

    mask_slice = ROI_mask[:, :, slice_idx]
    contours = find_contours(mask_slice, level=0.5)

    ax = axes[slice_idx]
    im = ax.imshow(slice_data.T, cmap='gray', origin='lower')
    ax.scatter(min_loc[0], min_loc[1], color='red', s=30)

    for contour in contours:
        ax.plot(contour[:, 0], contour[:, 1], color='red', linewidth=1)

    ax.set_title(f"Slice {slice_idx}")
    ax.axis("off")

#Turn off any unused subplots
for i in range(num_slices, len(axes)):
    axes[i].axis("off")

plt.tight_layout()
plt.show()

mean_vals = stats_df[stats_df['Mean Value'] != 0]['Mean Value'] 
average_mean = np.sum(mean_vals)/len(mean_vals)

## Plotting the trends (BoxPlot)

In [ ]:
Mean_10PP_mADT = np.array([]) #Array with the mean 10th percentile ADC before ADT for prostate gland
Mean_10PP_pADT = np.array([]) #Array with the mean 10th percentile ADC after ADT for prostate gland
Mean_10PT_mADT = np.array([]) #Array with the mean 10th percentile ADC before ADT for tumor lesion
Mean_10PT_pADT = np.array([]) #Array with the mean 10th percentile ADC after ADT for tumor lesion

Mib_stats = []
Mib_stats.append(('Prostate (-ADT)', Mean_10PP_mADT))
Mib_stats.append(('Prostate (+ADT)', Mean_10PP_pADT))
Mib_stats.append(('Tumor (-ADT)', Mean_10PT_mADT))
Mib_stats.append(('Tumor (+ADT)', Mean_10PT_pADT))

Mib_df = pd.DataFrame(Mib_stats, columns=['Group', 'Mean Intensity'])
print(Mib_df.head())

#Convert to long format
Mib_df_expanded = Mib_df.explode('Mean Intensity')
Mib_df_expanded = Mib_df_expanded.rename(columns={'Mean Intensity': 'ADC'})
Mib_df_expanded['ADC'] = Mib_df_expanded['ADC'].astype(float)

#Set a clear color palette
palette = {
    'Prostate (-ADT)': 'darkorange',
    'Prostate (+ADT)': 'peachpuff',
    'Tumor (-ADT)': 'royalblue',
    'Tumor (+ADT)': 'lightsteelblue'
}

plt.figure(figsize=(8, 6))
sns.boxplot(x='Group', y='ADC', data=Mib_df_expanded, palette=palette)
sns.swarmplot(x='Group', y='ADC', data=Mib_df_expanded, color='black', size=6, alpha=0.6)

plt.title("Mean ADC Values of the 10th Percentile Before and After ADT", fontsize=16)
plt.ylabel("ADC Normalized", fontsize=14)
plt.xlabel("", fontsize=14)
plt.xticks(rotation=15)
plt.tight_layout()
plt.show()